In [2]:
import matplotlib.pyplot as plt
from arch import arch_model
import pandas_ta
import pandas as pd
import numpy as np
import os
import yfinance as yf
from datetime import datetime, date, timedelta
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.signal import argrelextrema


In [ ]:
from dotenv import load_dotenv

load_dotenv()
# Retrieve the API key from the environment variable
databento_api_key = os.getenv("DATABENTO_API_KEY")
if not databento_api_key:
    raise ValueError("Please set the DATABENTO_API_KEY environment variable.")

In [ ]:
import databento as db
import time
from datetime import datetime, timedelta
import threading

# Global buffer to store incoming 1-minute records for the batch period
buffer = []

def handle_message(msg):
    """
    Callback function to process incoming messages.
    For example, this processes OHLCV messages.
    """
    if isinstance(msg, db.OhlcvMsg):
        record = {
            "ts_event": pd.to_datetime(msg.hd.ts_event, unit="ns"),
            "open": msg.open,
            "high": msg.high,
            "low": msg.low,
            "close": msg.close,
            "volume": msg.volume,
            "instrument_id": msg.hd.instrument_id,
        }
        buffer.append(record)
    else:
        # Handle other message types if needed.
        pass

def store_buffer():
    """
    Convert the buffered records to a DataFrame and store them (e.g. append to a CSV).
    Then clear the buffer.
    """
    global buffer
    if buffer:
        df_batch = pd.DataFrame(buffer)
        # Append to CSV file; if file doesn't exist, header is written automatically.
        # You can change the storage mechanism as needed (e.g., database insert).
        try:
            header = not pd.io.common.file_exists("5min_data.csv")
        except Exception:
            header = True
        df_batch.to_csv("5min_data.csv", mode="a", index=False, header=header)
        print(f"Stored batch of {len(buffer)} records at {datetime.utcnow()}")
        buffer = []

def run_batch_store():
    """
    Background thread that waits until the next 5-minute mark (UTC) to store the data.
    """
    while True:
        now = datetime.utcnow()
        # Compute the next 5-minute mark:
        # Floor current time to nearest 5-min mark and add 5 minutes.
        minutes = (now.minute // 5) * 5
        next_batch_time = now.replace(minute=minutes, second=0, microsecond=0) + timedelta(minutes=5)
        sleep_seconds = (next_batch_time - now).total_seconds()
        print(f"Sleeping for {sleep_seconds:.2f} seconds until next batch store at {next_batch_time} UTC.")
        time.sleep(sleep_seconds)
        store_buffer()

# Start the background storage thread (daemon thread so it exits with main thread)
storage_thread = threading.Thread(target=run_batch_store, daemon=True)
storage_thread.start()

# Create a live Databento client (replace with your actual API key)
client = db.Live(key=databento_api_key)

# Subscribe to 1-minute OHLCV data for ES futures.
client.subscribe(
    dataset="GLBX.MDP3",
    schema="OHLCV-1m",
    stype_in="parent",
    symbols="ES.FUT",
)

# Set the callback to process incoming messages.
client.add_callback(handle_message)

# Start the live stream (this call is blocking).
client.start()

Sleeping for 283.25 seconds until next batch store at 2025-03-04 18:20:00 UTC.


In [3]:
client.terminate()

In [ ]:
databento_api_key

In [5]:
for message in client:
    # Process each message (e.g., print or store it)
    print(message)

ValueError: Cannot start iteration after streaming has started, records may be missed. Don't call `Live.start` before iterating.